출처: https://blog.naver.com/PostView.naver?blogId=kunyoung90&logNo=222638574618&parentCategoryNo=&categoryNo=26&viewDate=&isShowPopularPosts=true&from=search

https://tjansry354.tistory.com/10


### 연관규칙분석이란?

연관분석이란 어떤 두 아이템 집합이 빈번히 발생하는 가를 알려주는 일련의 규칙들을 생성하는 알고리즘.

"고객이 와인을 구매한다면 그 고객의 15%가 치즈를 구매한다"라고 하는 것이 일종의 연관규칙이 됨.

* 지지도(support): 전체 거래 중 A와 B가 동시에 포함된 거래의 비율 (A->B)
  * 빵 -> 버터의 지지도: (빵과 버터를 동시에 포함한 거래 수) / (전체 거래 수)
* 신뢰도(confidence): 품목 A를 구매했을 때 B도 구매했을 확률
  * 빵 -> 버터의 신뢰도: (빵과 버터를 동시에 포함한 거래 수) / (빵이 포함된 거래 수)
* 향상도(lift): A를 구매했을 때 B를 구매할 가능성에 관한 지표로 향상도가 1이라면 A와 B가 확률적으로 독립에 가까움을 의미. 향상도가 1보다 크다면 A를 구매했을 때 B를 구매할 가능성이 높아지고, 반대로 1보다 작다면 A를 구매했을 때 B를 구매할 가능성이 낮아지게 됨.
  * 빵 -> 버터의 향상도: ((빵,버터의 동시 거래 수) x (전체 거래 수)) / (A포함 거래 수 x B포함 거래 수) 
  * 신뢰도 / 지지도

----------------------
#### 예시(Apriori)

In [4]:
import pandas as pd
df = pd.read_csv(r'C:\Users\whgur\Desktop\workspace\ADP\data\chipotle.csv')
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


apriori알고리즘을 사용하려면 같은 트랜잭션의 아이템끼리 list 자료형식으로 구성해야 한다.([[딸기, 배],[사과, 딸기]] 처럼)

In [13]:
items = df.groupby('order_id').apply(lambda x: [i for i in x['item_name']]).values.tolist()
items[:5]

[['Chips and Fresh Tomato Salsa',
  'Izze',
  'Nantucket Nectar',
  'Chips and Tomatillo-Green Chili Salsa'],
 ['Chicken Bowl'],
 ['Chicken Bowl', 'Side of Chips'],
 ['Steak Burrito', 'Steak Soft Tacos'],
 ['Steak Burrito', 'Chips and Guacamole']]

In [22]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

Encoder = TransactionEncoder()
TR_array = Encoder.fit_transform(items)
TR_array

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [21]:
df_item = pd.DataFrame(TR_array, columns=Encoder.columns_)
df_item.head()

,6 Pack Soft Drink,Barbacoa Bowl,Barbacoa Burrito,Barbacoa Crispy Tacos,Barbacoa Salad Bowl,Barbacoa Soft Tacos,Bottled Water,Bowl,Burrito,Canned Soda,...,Steak Crispy Tacos,Steak Salad,Steak Salad Bowl,Steak Soft Tacos,Veggie Bowl,Veggie Burrito,Veggie Crispy Tacos,Veggie Salad,Veggie Salad Bowl,Veggie Soft Tacos
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [5]:
item_sets = apriori(df_item, min_support=0.05, use_colnames=True)
item_sets

# FP-Growth
#Apriori와의 차이는 FP-Tree를 생성한 후에 최소 지지도 이상의 패턴만을 추출한다는 것이다. FP tree는 아이템 별로 노드를 추가하면서 트리를 생성하는 과정이다.
# from mlxtend.frequent_patterns import fpgrowth
# fpgrowth(df, min_support=0.5, use_colnames=True)

,support,itemsets
0,0.083969,(Bottled Water)
1,0.051254,(Canned Soda)
2,0.150491,(Canned Soft Drink)
3,0.335333,(Chicken Bowl)
4,0.266630,(Chicken Burrito)
5,0.053435,(Chicken Salad Bowl)
6,0.058342,(Chicken Soft Tacos)
7,0.113413,(Chips)
8,0.059978,(Chips and Fresh Tomato Salsa)
9,0.258451,(Chips and Guacamole)


In [31]:
association_rules(item_sets, metric="lift", min_threshold=1)

#association_rules(item_sets, metric="confidence", min_threshold=0.005)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Canned Soft Drink),(Chicken Bowl),0.150491,0.335333,0.060523,0.402174,1.199328,0.010059,1.111807
1,(Chicken Bowl),(Canned Soft Drink),0.335333,0.150491,0.060523,0.180488,1.199328,0.010059,1.036604
2,(Chips),(Chicken Bowl),0.113413,0.335333,0.066521,0.586538,1.749124,0.028490,1.607568
3,(Chicken Bowl),(Chips),0.335333,0.113413,0.066521,0.198374,1.749124,0.028490,1.105985


antecedents : 조건절

consequents : 결과절

그럼 이제 손님이 "어떤 메뉴를 주로 먹나요?" 에 대해,

'Chicken Bowl'을 먹는다면 'Canned Soft Drink'를 함께 먹거나, 'Chicken Bowl'을 먹을 때 'Chips'를 주로 함께 먹는다고 말 할 수 있다.